# PyTorch Neural-Transfert tutorial

## Introduction

Welcome! This tutorial explains how to impletment the [Neural-Style](https://arxiv.org/abs/1508.06576) algorithm devloped by Leon A. Gatys, Alexander S. Ecker and Matthias Bethge.

### Neural what?

The Neural-Style, or Neural-Transfert, is an algorithm that taks as input a content-image (e.g. a tortle), a style-image (e.g. artistic waves) and return the content of the content-image as if it was 'painted' using the artistic style of the style-image:

![content1](images/neuralstyle.png)

### How does it work?

The principe is simple: we define two distances, one for the content ($D_C$) and one for the style ($D_S$). $D_C$ measues how different is the content between two images, while $D_S$ measures how different is the style between two images. Then, we just try to minimise those two distances. 

### OK. How does it work?

Well, going further requires some mathematics. Let $C_{nn}$ be a pre-trained deep convolutional neural network and $X$ be any image. $C_{nn}(X)$ is the network fed by $X$ (containing feature maps at all layers). Let $F_{XL} \in C_{nn}(X)$ be the feature maps at depth layer $L$, all vectorized and concatenated in one single vector. We simply define the content of $X$ at layer $L$ by $F_{XL}$. Then, if $Y$ is another image of same the size than $X$, we define the distance of content at layer $L$ as follow:

$$D_C^L(X,Y) = \|F_{XL} - F_{YL}\|^2 = \sum_i (F_{XL}(i) - F_{YL}(i))^2$$

Where $F_{XL}(i)$ is the $i^{th}$ element of $F_{XL}$. The style is a bit less trivial to define. Let $F_{XL}^k$ with $k \leq K$ be the vectorized $k^{th}$ of the $K$ feature maps at layer $L$. The style $G_{XL}$ of $X$ at layer $L$ is defined by the Gram produce of all vectorized feature maps $F_{XL}^k$ with $k \leq K$. In other words, $G_{XL}$ is a $K$x$K$ matrix and the element $G_{XL}(k,l)$ at the $k^{th}$ line and $l^{th}$ column of $G_{XL}$ is the vectorial produce between  $F_{XL}^k$ and $F_{XL}^l$:

$$G_{XL}(k,l) = \langle F_{XL}^k, F_{XL}^l\rangle = \sum_i F_{XL}^k(i) . F_{XL}^l(i)$$

Where $F_{XL}^k(i)$ is the $i^{th}$ element of $F_{XL}^k$. We can see $G_{XL}(k,l)$ as a measure of the correlation between feature maps $k$ and $l$. In that way, $G_{XL}$ represents the correlation matrix of feature maps of $X$ at layer $L$. Note that the size of $G_{XL}$ only depends on the number of feature maps, not on the size of $X$. Then, if $Y$ is another image $\textit{of any size}$, we define the distance of style at layer $L$ as follow:

$$D_S^L(X,Y) = \|G_{XL} - G_{YL}\|^2 = \sum_{k,l} (F_{XL}(k,l) - F_{YL}(k,l))^2$$

In order to minimize in one shot $D_C(X,C)$ between a variable image $X$ and target content-image $C$ and $D_S(X,S)$ between $X$ and target style-image $S$, both computed at several layers , we compute and sum the gradients (derivative with respect to $X$) of each distance at each wanted layer:

$$\nabla_{\textit{total}}(X,S,C) = \sum_{L_C} w_{CL_C}.\nabla_{\textit{content}}^{L_C}(X,C). \sum_{L_S} w_{SL_S}.\nabla_{\textit{style}}^{L_S}(X,S)$$

Where $L_C$ and $L_S$ are respectivement the wanted layers (arbitrary stated) of content and style and $w_{CL_C}$ and $w_{SL_S}$ the weights (arbitrary stated) associated with the style or the content at each wanted layer. Then, we run a gradient descent over $X$:

$$X \leftarrow X - \alpha \nabla_{\textit{total}}(X,S,C)$$

Ok. That's enough with maths. If you want to go deeper (how to compute the gradients) $\textbf{we encourage you to read the original paper}$ by Leon A. Gatys and AL, where everything is much better and much clearer explained. For our implementation in PyTorch, we already have evrything we need: indeed, with PyTorch, all the gradients are automatically and dynamically computed for you (while you use functions from the library). This is why the implementation of this algorithm becomes very confortable with PyTorch. 

## PyTorch implementation

If you are not sure to understand all the mathematics above, you will probably understand by implementing it. If you are discovering PyTorch, we recommend you to first read the [Introduction to PyTorch](https://github.com/pytorch/tutorials/blob/master/Deep%20Learning%20with%20PyTorch.ipynb).

### Packages

We will have recourse to the following packages:

* `torch`, `torch.nn`, `numpy` (indispensables packages for neural networks with PyTorch)
* `torch.autograd.Variable` (dynamic computation of the gradient wrt a variable)
* `torch.optim` (efficient gradient descents)
* `PIL`, `PIL.Image`, `matplotlib.pyplot` (load and display images)
* `torchvision.transforms` (treat PIL images and transform into torch tensors)
* `torchvision.models` (train or load pre-trained models)

In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim

import PIL
from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torchvision.models as models

### Cuda

In this tutorial, we are assuming you have a GPU with cuda installed and that you want to use it. If you dont, just remove all the ''`.cuda()`'' and the ''`.cpu()`'' in the exemple codes. They are used to moove allocated proccesses associated with a variable (or a torch module) from the CPU to the GPU and vice versa.

### Load images

In order to simplify the implementation, let's start by importing a style and a content image of the same dimentions. We then scale them to the desired output image size (200 in the example) and transform them into torch tensors, ready to feed a neural network:

In [ ]:
imsize = 200 # desired size of the output image

loader = transforms.Compose([
            transforms.Scale(imsize), # scale imported image
            transforms.ToTensor()]) # transform it into a torch tensor

def image_loader(image_name):
    image = Image.open(image_name)
    image = Variable(loader(image))
    image = image.unsqueeze(0) # fake batch dimension required to fit network's input dimensions
    return image

style = image_loader("images/style.jpg").cuda()
content = image_loader("images/content.jpg").cuda()

assert style.data.size() == content.data.size(), "we need to import style and content images of the same size"

Imported PIL images has values between 0 and 255. Transformed into torch tensors, their values are between 0 and 1. This is an important detail: neural networks from torch library are trained with 0-1 tensor image. If you try to feed the networks with 0-255 tensor images the activated feature maps will have no sens. This is not the case with pre-trained networks from the Caffe library: they are trained with 0-255 tensor images.

### Display images

We will use `plt.imshow` to display images. So we need to first reconvert them into PIL images:

In [ ]:
unloader = transforms.ToPILImage() # reconvert into PIL image

def imshow(tensor):
    image = tensor.clone().cpu() # we clone the tensor in order to not do changes on it
    image.resize_(3,imsize,imsize) # remove the fake batch dimension
    image = unloader(image)
    plt.imshow(image)
    
fig = plt.figure()

plt.subplot(221)
imshow(style.data)
plt.subplot(222)
imshow(content.data)

### Content loss

The content loss is a function that takes as input the feature maps $F_{XL}$ at a layer $L$ in a network fed by $X$ and return the weigthed content distance $w_{CL}.D_C^L(X,C)$ between this image and the content image. Hence, the weight $w_{CL}$ and the target content $F_{CL}$ are parameters of the function. We implement this function as a torch module with a constructor that takes these parameters as input. The distance $\|F_{XL} - F_{YL}\|^2$ is the Mean Square Error between the two sets of feature maps, that can be computed using a criterion `nn.MSELoss` stated as a third parameter.

We will add our content losses at each desired layer as additive modules of the neural network. That way, each time we will feed the network with an input image $X$, all the content losses will be computed at the desired layers and, thanks to autograd, all the gradients will be computed. For that, we just need to make the `forward` method of our module returning the input: the module becomes a ''transparent layer'' of the neural network. The computed loss is saved as a parameter of the module.

Finally, we define a fake `backward` method, that just call the backward method of `nn.MSELoss` in order to reconstruct the gradient. This method returns the computed loss: this will be usefull when running the gradien descent in order to display the evolution of style and content losses.

In [ ]:
class ContentLoss(nn.Module):
    def __init__(self, target, weight):
        super(ContentLoss, self).__init__()
        self.target = target.detach() * weight
        self.weight = weight
        self.criterion = nn.MSELoss()

    def forward(self, input):
        self.loss = self.criterion.forward(input*self.weight, self.target)
        self.output = input
        return self.output

    def backward(self, retain_variables=True):
        self.loss.backward(retain_variables=retain_variables)
        return self.loss.data[0]

$\textbf{Important detail}$: This module, although it is named `ContentLoss`, is not a true PyTorch Loss function. If you want to define your content loss as a PyTorch Loss, you have to create a PyTorch autograd Function and to recompute/implement the gradient by the hand in the `backward` method. 